In [1]:
import numpy

import matplotlib.pyplot as plt

from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import KFold
import time
import math
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import itertools
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
# fix random seed for reproducibility
numpy.random.seed(7)

C:\Users\engrl\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


# Tips of working of evolutionary_search library
1. It did not work. Started working normally when I installed another library i.e. Neuro_evolution which is keras based and work for holdout.

In [2]:
#Importing the Train Data 
import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)
print(X[[1039]])

X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]
[[1.809000e+00 1.485100e-04 6.800000e-01 8.430000e-01 2.040000e+00
  7.881000e+00 7.820000e-01 2.690000e+00 4.543000e+00 1.107300e+01
  8.069000e+00 9.255540e-01 9.748100e-02 1.347200e+01 1.192600e+02
  1.216300e+02 8.028000e+00 1.081440e+02 1.375460e+02 6.200000e+01
  6.000000e+01 8.211245e-03 5.658130e-04 1.818200e+01 1.000000e+00
  3.387000e+00]]


# Below code extracts vowel u phonations

In [4]:
i=1
indices = 2 #if you change this to 1 or 2, also change the increment to 27 and 28, respectively
increment=28
while i<=39:
    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y = X_train_withlabels[:, 26]
X = numpy.delete(X_train_withlabels, 26, 1)
print(X.shape)
print(Y.shape)

(40,)
(40, 27)
(40, 26)
(40,)


# Below Code is for Genetic Algorithm Code.

In [5]:
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

import itertools
random.seed(1)


X_Origionl = X
y_Origionl = Y

x = range(1,101)
lda = LDA(n_components=1)
X = lda.fit_transform(X_Origionl, y_Origionl)
y = y_Origionl
paramgrid = {"hidden_layer_sizes": list(itertools.combinations(x,2)),
             "solver" :['lbfgs']
            
             }
print("Size: ", len(paramgrid))

Size:  2


In [6]:
time_a = time.time()
if __name__=="__main__":
    #pool = Pool(4)
    cv = EvolutionaryAlgorithmSearchCV(estimator=MLPClassifier(),
                                       params=paramgrid,
                                       scoring="accuracy",
                                       cv=KFold(n_splits=40),
                                       verbose=True,
                                       population_size=15,
                                       gene_mutation_prob=0.10,
                                       tournament_size=3,
                                       generations_number=10)
                                       #pmap = pool.map)
    %time cv.fit(X, y)
time_b = time.time()
print("Time Elapsed =", time_b-time_a)

Types [1, 1] and maxint [4949, 0] detected
--- Evolve in 4950 possible combinations ---
gen	nevals	avg     	min	max 	std      
0  	15    	0.946667	0.9	0.95	0.0124722
1  	9     	0.95    	0.95	0.95	3.33067e-16
2  	12    	0.95    	0.95	0.95	3.33067e-16
3  	6     	0.95    	0.95	0.95	3.33067e-16
4  	10    	0.95    	0.95	0.95	3.33067e-16
5  	11    	0.95    	0.95	0.95	3.33067e-16
6  	7     	0.95    	0.95	0.95	3.33067e-16
7  	8     	0.95    	0.95	0.95	3.33067e-16
8  	11    	0.95    	0.95	0.95	3.33067e-16
9  	5     	0.95    	0.95	0.95	3.33067e-16
10 	9     	0.95    	0.95	0.95	3.33067e-16
Best individual is: {'hidden_layer_sizes': (12, 79), 'solver': 'lbfgs'}
with fitness: 0.95
Wall time: 9 s
Time Elapsed = 8.995530605316162


# Time complexity analysis with grid search

In [6]:
lda = LDA(n_components=1)
X_new = lda.fit_transform(X, Y)

Best_Acc =0

time_a = time.time()
Nodes = range(1,101)
for nodes1 in Nodes:
    for nodes2 in Nodes:
        i=0
        Net_Acc=0
        while i<=39:

            #########Setting Train Test Parts for Each Subject
            X_test = X_new[[i]]
            X_train = numpy.delete(X_new, i, 0)
            Y_train = numpy.delete(Y, i, 0)
            Y_test = Y[[i]]


            ##############Models Part##############################################
            #Model Fiting
            # checkpoint
            model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(nodes1, nodes2,),random_state=1)
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)
            # evaluate the model
            scores = accuracy_score(Y_test, Y_pred)
            #print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


            if scores==1:
                Net_Acc = Net_Acc+1

            i = i+1
        Acc = (Net_Acc/40)*100
        
        if (Acc>Best_Acc):
            Best_Acc = Acc
            print("Best Acc ====================================", Best_Acc)
            print("Nodes1 =", nodes1, "Nodes2=", nodes2)
time_b = time.time()
print("Time Elapsed =", time_b-time_a)

Number of LDA Selected Features =================== (40, 1)
Best Acc ==================================== 95.0
Nodes1 = 1 Nodes2= 4
Best Acc ==================================== 97.5
Nodes1 = 1 Nodes2= 13
Time Elapsed = 5149.544309139252


# Regenerating Results of Table III of the Paper

In [7]:
#Importing the Train Data 
import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)

X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]


In [8]:
#Extracting Vowel "u" Phonations.
i=1
indices = 2 #if you change this to 1 and increment to 27, we will get Vowel "o" Dataset
increment=28
while i<=39:
    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y = X_train_withlabels[:, 26]
X = numpy.delete(X_train_withlabels, 26, 1)
print(X.shape)
print(Y.shape)
print(indices)
print(Y)

(40,)
(40, 27)
(40, 26)
(40,)
[   2   28   54   80  106  132  158  184  210  236  262  288  314  340
  366  392  418  444  470  496  522  548  574  600  626  652  678  704
  730  756  782  808  834  860  886  912  938  964  990 1016]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
lda = LDA(n_components=1)
X_new = lda.fit_transform(X, Y)

Best_Acc =0
TP=0
TN=0
FP=0
FN=0


i=0
Net_Acc=0
while i<=39:

    #########Setting Train Test Parts for Each Subject
    X_test = X_new[[i]]
    X_train = numpy.delete(X_new, i, 0)
    Y_train = numpy.delete(Y, i, 0)
    Y_test = Y[[i]]


    ##############Models Part##############################################
    #Model Fiting
    # checkpoint
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 29,),random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    # evaluate the model
    scores = accuracy_score(Y_test, Y_pred)
    
    if i<=19:
        if scores==1:
            TP = TP+1 #This is for evaluationg sensitivity
    if i>19:
        if scores==1:
            TN = TN+1  #This is for evaluating specificity
    

    if scores==1:
        Net_Acc = Net_Acc+1 #This is for evaluationg overall accuracy

    i = i+1
Acc = (Net_Acc/40)*100
FN=20-TP
FP=20-TN

Best_Acc = Acc
print("Acc ====================================", Best_Acc)
print("Sensitivity =", (TP/(20))*100)
print("Specificity =", (TN/(20))*100)
print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Acc ==================================== 95.0
Sensitivity = 95.0
Specificity = 95.0
Mathew Correlation Coefficients MCC= 0.9


# Regenerating Results of Table IV

In [10]:
#Importing the data of Training Database
import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)


X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]


In [11]:
#Extracting Vowel "o" Phonations.
i=1
indices = 1 #if you change this to 1 and increment to 27, we will get Vowel "o" Dataset
increment=27
while i<=39:
    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y = X_train_withlabels[:, 26]
X = numpy.delete(X_train_withlabels, 26, 1)
print(X.shape)
print(Y.shape)
print(indices)
print(Y)

(40,)
(40, 27)
(40, 26)
(40,)
[   1   27   53   79  105  131  157  183  209  235  261  287  313  339
  365  391  417  443  469  495  521  547  573  599  625  651  677  703
  729  755  781  807  833  859  885  911  937  963  989 1015]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
lda = LDA(n_components=1)
X_new = lda.fit_transform(X, Y)

Best_Acc =0
TP=0
TN=0
FP=0
FN=0


i=0
Net_Acc=0
while i<=39:

    #########Setting Train Test Parts for Each Subject
    X_test = X_new[[i]]
    X_train = numpy.delete(X_new, i, 0)
    Y_train = numpy.delete(Y, i, 0)
    Y_test = Y[[i]]


    ##############Models Part##############################################
    #Model Fiting
    # checkpoint
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 29,),random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    # evaluate the model
    scores = accuracy_score(Y_test, Y_pred)
    
    if i<=19:
        if scores==1:
            TP = TP+1
    if i>19:
        if scores==1:
            TN = TN+1
    

    if scores==1:
        Net_Acc = Net_Acc+1

    i = i+1
Acc = (Net_Acc/40)*100
FN=20-TP
FP=20-TN

Best_Acc = Acc
print("Acc ====================================", Best_Acc)
print("Sensitivity =", (TP/(20))*100)
print("Specificity =", (TN/(20))*100)
print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Acc ==================================== 92.5
Sensitivity = 95.0
Specificity = 90.0
Mathew Correlation Coefficients MCC= 0.8510644963469901


# Regenerating Results of Table V

In [13]:
#Importing the data of Training Database
import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)


X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]


In [14]:
#Extracting Vowel "a" Dataset
i=1
indices = 0 #if you change this to 1 and increment to 27, we will get Vowel "o" Dataset
increment=26
while i<=39:
    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y = X_train_withlabels[:, 26]
X = numpy.delete(X_train_withlabels, 26, 1)
print(X.shape)
print(Y.shape)
print(indices)

(40,)
(40, 27)
(40, 26)
(40,)
[   0   26   52   78  104  130  156  182  208  234  260  286  312  338
  364  390  416  442  468  494  520  546  572  598  624  650  676  702
  728  754  780  806  832  858  884  910  936  962  988 1014]


In [15]:
lda = LDA(n_components=1)
X_new = lda.fit_transform(X, Y)

Best_Acc =0
TP=0
TN=0
FP=0
FN=0


i=0
Net_Acc=0
while i<=39:

    #########Setting Train Test Parts for Each Subject
    X_test = X_new[[i]]
    X_train = numpy.delete(X_new, i, 0)
    Y_train = numpy.delete(Y, i, 0)
    Y_test = Y[[i]]


    ##############Models Part##############################################
    #Model Fiting
    # checkpoint
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 29,),random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    # evaluate the model
    scores = accuracy_score(Y_test, Y_pred)
    
    if i<=19:
        if scores==1:
            TP = TP+1
    if i>19:
        if scores==1:
            TN = TN+1
    

    if scores==1:
        Net_Acc = Net_Acc+1

    i = i+1
Acc = (Net_Acc/40)*100
FN=20-TP
FP=20-TN

Best_Acc = Acc
print("Acc ====================================", Best_Acc)
print("Sensitivity =", (TP/(20))*100)
print("Specificity =", (TN/(20))*100)
print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Acc ==================================== 92.5
Sensitivity = 95.0
Specificity = 90.0
Mathew Correlation Coefficients MCC= 0.8510644963469901


# Regenerating Results of Table VI

In [16]:
# load pima indians dataset
#Importing the Train Data 
import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)

X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]


In [17]:
#Extracting vowel data from training database
i=2
indices = numpy.array(range(0, 2))
lb = 26
ub = 28
while i<=40:
    indices = numpy.append(indices, range(lb, ub))    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    lb = lb+26
    ub = ub+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y_train = X_train_withlabels[:, 26]
X_train = numpy.delete(X_train_withlabels, 26, 1)
print(X_train.shape)
print(Y_train.shape)

(80,)
(80, 27)
(80, 26)
(80,)


In [18]:
#Importing the Test Data 
df = pd.read_csv("E:\PD Dataset From Laptop/test_data.txt")
X_first = numpy.array(df)
X_test_withlabels = numpy.delete(X_first, 0,1)
X_test = numpy.delete(X_test_withlabels, 26, 1)
print("X_with contains 26 input features and class label but not with patient id", X_test_withlabels.shape)   
print(X_test.shape)
Y_test = X_test_withlabels[:, 26]
print(Y_test.shape)

X_with contains 26 input features and class label but not with patient id (168, 27)
(168, 26)
(168,)


In [19]:
lda = LDA(n_components=1)
X_FS_train = lda.fit_transform(X_train, Y_train)


######It is important to note below that we are using LDA model fitted on Training Data and Reduce the testing data then.
lda = LDA(n_components=1)
X_FS_test = lda.fit(X_train, Y_train).transform(X_test)



In [20]:
########### LOSO on Testing Database##############################

Best_Acc=0

Net_Acc=0
i=1
lb=0
ub=6
while i<=28:

    #########Setting Train Test Parts for Each Subject
    X_test_subj = X_FS_test[range(lb,ub)]
    Y_test_subj = Y_test[range(lb,ub)]
    #print(X_test_subj.shape)
    #print(Y_test_subj.shape)

    ##############Models Part##############################################
    #Model Fiting
    # checkpoint
    ##############Models Part##############################################

    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 29,), random_state=1)
    model.fit(X_FS_train, Y_train)
    Y_pred = model.predict(X_test_subj)
    scores = accuracy_score(Y_test_subj, Y_pred)
    if scores>=0.5:
        Net_Acc = Net_Acc+1


    lb=lb+6
    ub=lb+6
    i = i+1
Acc = (Net_Acc*100)/28

Best_Acc = Acc
print("Best Acc =", Best_Acc)

Best Acc = 100.0
